In [19]:
def can_cover_with_balls(tree, k):

    # tree: dictionary representing the tree
    # k: maximum radius 

    # Function to determine the maximum depth of the tree
    def get_max_depth(node):

        # node: starting node

        if node not in tree or not tree[node]:
            return 1
        return 1 + max(get_max_depth(child) for child in tree[node])
    
    # Perform DFS to return the covered nodes
    def dfs(node, radius):

        if radius == 0:
            return set()
        
        covered = {node}

        if node in tree:
            for child in tree[node]:
                # recursively
                covered.update(dfs(child, radius - 1))

        return covered

    # Possibility to cover the tree 
    def can_cover(node, radius, remaining_balls):

        # node: starting point
        # radius: given radius
        # ramaining_balls
   
        if not remaining_balls:
            return False
        
        if radius == 0:
            return len(remaining_balls) == 0
        
        # Get the nodes covered by the current ball and the remaining nodes
        covered_nodes = dfs(node, radius)
        remaining_nodes = all_nodes - covered_nodes

        # Can this remaining nodes be covered by the remaining balls ?

        if not remaining_nodes:
            return True
        
        for next_ball in remaining_balls:

            for next_node in remaining_nodes:

                # recursively
                if can_cover(next_node, next_ball, remaining_balls - {next_ball}):
                    return True
                
        return False

    # get all the nodes of the tree
    all_nodes = set(tree.keys())
    for key in tree.keys():
        all_nodes.update(tree[key])

    # check all the nodes for coverage
    for start_node in all_nodes:
        if can_cover(start_node, k, set(range(1, k+1))):
            return True
    return False

# Example usage:
# Create a sample Galton-Watson tree as a dictionary
tree = {
    0: [1, 2],
    1: [3],
    2: [0],
    3: [1]
}

k = 3
print(can_cover_with_balls(tree, k))  # Output: True or False depending on the tree structure


True


In [20]:
import math
import numpy as np

def burning_radius_number(tree):
    n = len(tree)
    k = math.ceil(np.sqrt(n))

    while can_cover_with_balls(tree, k) is True:
        if can_cover_with_balls(tree, k - 1) is True:
            k -= 1
        else:
            return k
        
    return math.ceil(np.sqrt(n))


# Tim's Code:

In [21]:

import random



#first initialise the offspring distribution (zeta): p_0, p_1, ... p_k
def zeta_unspecified(randweight=10, tolerance=10**(-6)):
    T = 1
    p = []
    while T > 0:
        newp = random.random()/randweight
        if T >= newp:
            T = T - newp
            p.append(newp)
        else:
            p.append(T)
            T = 0
    assert(-tolerance <= sum(p) - 1 <= tolerance)
    return p


# a zeta where the expected number of children is 1, this is called a critical GW process
zeta_critical_example1 = [0.4, 0.3, 0.2, 0.1]
zeta_critical_example2 = [0.3, 0.4, 0.3]

##########################################################################################

#next we use the previous code to make a zeta and generate a GW tree of specified size n
def generate_GW(zeta, n):
    kidlist = [i for i in range(len(zeta))]
    edges = set()
    parent=0
    upper=1
    while upper <= n-1:
        [nchild] = random.choices(kidlist,weights=zeta, k=1)
        #print(nchild)
        for _ in range(nchild):
            edges.add((parent, upper))
            upper += 1
            #print(edges)
            if upper >= n:
                return (n, edges)
        if parent+1 >= upper:
            return (n, edges)
        parent += 1



# Brute Force Burning Number:

In [22]:
def burn_tree_recursive(tree, burning, burnt, rounds, memo):
    
    # tree : The adjacency list representation of the tree.
    # burning : The set of nodes that are currently burning.
    # burnt : The set of nodes that have already burnt down.
    # rounds : The number of rounds that have passed.
    # memo : A dictionary used for memoization to store already computed results.
    
    # Have we already parsed through this?
    burnt_tuple = tuple(burnt)
    if burnt_tuple in memo:
        return memo[burnt_tuple]
    
    # Did we burn the whole tree?
    if len(burnt) == len(tree):
        return rounds

    # New burning nodes of this round
    new_burning = []
    for node in burning:
        for neighbor in tree[node]:
            if neighbor not in burnt:
                new_burning.append(neighbor)
    
    # Update the lists
    burning.extend(new_burning)
    burnt.extend(new_burning)
    rounds += 1

    # Have we burnt the whole tree?
    if len(burnt) == len(tree):
        memo[burnt_tuple] = rounds
        return rounds

    min_rounds = float('inf')
    # Check for minimum when starting with this source node
    for new_node in tree:
        if new_node not in burnt:

            burning.append(new_node)
            burnt.append(new_node)

            # burn the remaining nodes
            result = burn_tree_recursive(tree, burning.copy(), burnt.copy(), rounds, memo)
            min_rounds = min(min_rounds, result)

            # update
            burning.remove(new_node)
            burnt.remove(new_node)

    memo[burnt_tuple] = min_rounds
    # print(memo)
    return min_rounds

# memo : memorize the rounds needed to burn a sequence of nodes


def burning_number(tree):

    min_burning_number = float('inf')
    memo = {}
    nodes = list(tree.keys())

    if len(tree) == 0:
        return 0

    # Iterate through the nodes
    for node in nodes:

        burning = [node]
        burnt = [node]

        # get the minimum rounds for that node
        result = burn_tree_recursive(tree, burning, burnt, 1, memo)

        # update the burning number
        min_burning_number = min(min_burning_number, result)

    return min_burning_number



In [23]:
from collections import defaultdict, deque
from math import ceil, sqrt

def create_tree(nodes, edges):
    tree = defaultdict(list)
    for u, v in edges:
        tree[u].append(v)
        tree[v].append(u)
    return tree

### Plotting trees:

In [24]:
import matplotlib.pyplot as plt

# Define the functions from the last cell
def plot_tree(edges):
    def get_positions(node, depth=0, pos={}, x=0, dx=1):
        if node not in pos:
            pos[node] = (x, -depth)
            children = [v for u, v in edges if u == node]
            if children:
                dx = dx / len(children)
                next_x = x - dx * (len(children) - 1) / 2
                for i, child in enumerate(children):
                    pos = get_positions(child, depth + 1, pos, next_x + i * dx, dx)
        return pos

    def draw_tree(ax, edges, pos):
        for u, v in edges:
            x_values = [pos[u][0], pos[v][0]]
            y_values = [pos[u][1], pos[v][1]]
            ax.plot(x_values, y_values, 'gray')

        for node, (x, y) in pos.items():
            ax.scatter(x, y, c='skyblue', s=100)
            ax.text(x, y, str(node), fontsize=12, ha='center', va='center')

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_aspect('equal')
    ax.set_axis_off()

    root = 0
    pos = get_positions(root)
    draw_tree(ax, edges, pos)

    plt.show()

## Checking the burning number:

In [25]:
zeta = [0.1, 0, 0.2, 0.5, 0.1]

n = 7

for _ in range(2000):
    zeta = [0.1, 0.3, 0.5, 0.1]
    nodes, edges = generate_GW(zeta, int(n))
    n = nodes
    nodes = list(range(nodes))
    tree = create_tree(nodes, edges)
    if burning_number(tree) != burning_radius_number(tree):
        plot_tree(edges)
        print(f'Burning number is {burning_number(tree)}')
        print(f'Radius functions give {burning_radius_number(tree)}')